In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import shapely
import shapely.ops

import pydicom

import pymedphys

In [ ]:
dcm_path = pymedphys.data_path("dedupe-contour.dcm")

In [ ]:
CONTOUR_GEOMETRIC_TYPE = 'CLOSED_PLANAR'

In [ ]:
dcm = pydicom.read_file(str(dcm_path), force=True)

In [ ]:
contour_sequences = dcm.ROIContourSequence

In [ ]:
current_contour_sequence = contour_sequences[1]

In [ ]:
current_contour_sequence

In [ ]:
current_contour_sequence.ContourSequence[0]

In [ ]:
# current_contour_sequence

In [ ]:
contours_by_z = {}
image_sequence_by_z = {}

expected_contour_keys = {
    pydicom.tag.Tag(*tag)
    for tag in [(0x3006, 0x0050), (0x3006, 0x0046), (0x3006, 0x0042), (0x3006, 0x0016)]
}

for contour in current_contour_sequence.ContourSequence:
    if contour.ContourGeometricType != CONTOUR_GEOMETRIC_TYPE:
        raise ValueError(f"Only {CONTOUR_GEOMETRIC_TYPE} type is supported")
        
    if set(contour.keys()) != expected_contour_keys:
        raise ValueError("Unexpected contour sequence format")
        
    contour_data = contour.ContourData
        
    x = np.array(contour_data[0::3])
    y = np.array(contour_data[1::3])
    z = np.array(contour_data[2::3])
    
    unique_z = np.unique(z)
    
    if len(unique_z) != 1:
        raise ValueError("All z values should be equal")
        
    z = unique_z[0]
    polygon = shapely.geometry.Polygon(zip(x, y))
    
    try:
        contours_by_z[z].append(polygon)
    except KeyError:
        contours_by_z[z] = [polygon]
        
    try:
        image_sequence_by_z[z].append(contour.ContourImageSequence)
    except KeyError:
        image_sequence_by_z[z] = [contour.ContourImageSequence]

In [ ]:
image_sequence_by_z_collapsed = {}

expected_image_sequence_keys = {
    pydicom.tag.Tag(*tag)
    for tag in [(0x0008, 0x1150), (0x0008, 0x1155)]
}

for z, image_sequences in image_sequence_by_z.items():
    values = set()
    
    for image_sequence in image_sequences:
        if len(image_sequence) != 1:
            raise ValueError("Expected only one item per image sequence")
            
        image_sequence_item = image_sequence[0]
        
        if set(image_sequence_item.keys()) != expected_image_sequence_keys:
            raise ValueError("Unexpected contour image sequence format")
            
        values.add((
            image_sequence_item.ReferencedSOPClassUID,
            image_sequence_item.ReferencedSOPInstanceUID,
        ))
    
    if len(values) != 1:
        raise ValueError("Each z value should only point to one image slice")
        
    image_sequence_by_z_collapsed[z] = image_sequences[0]

In [ ]:
all_merged = {}

for z, contours in contours_by_z.items():
    all_merged[z] = shapely.ops.unary_union(contours)

In [ ]:
all_merged

In [ ]:
item = all_merged[-100]

In [ ]:
item[0].exterior.coords.xy

In [ ]:
isinstance(item, shapely.geometry.multipolygon.MultiPolygon)

In [ ]:
def get_coords_from_polygon(polygon):
    return polygon.exterior.coords.xy

In [ ]:
def get_coords_from_multipolygon(multipolygon):
    return [
        get_coords_from_polygon(item) for item in multipolygon
    ]

In [ ]:
def get_coords_from_polygon_or_multipolygon(polygon_or_multipolygon):
    try:
        return [get_coords_from_polygon(polygon_or_multipolygon)]
    except AttributeError:
        return get_coords_from_multipolygon(polygon_or_multipolygon)

In [ ]:
dicom_format_coords_by_z = {}

for z, merged in all_merged.items():
    coords = get_coords_from_polygon_or_multipolygon(merged)
    new_contour_data = []
    for coord in coords:
        stacked_coords = np.hstack(list(zip(coord[0], coord[1], z * np.ones_like(coord[1]))))
        stacked_coords = np.round(stacked_coords, 1)
        stacked_coords = stacked_coords.tolist()
        
        print(stacked_coords)
        
        new_contour_data.append(stacked_coords)
        
    dicom_format_coords_by_z[z] = new_contour_data

In [ ]:
new_contour_sequence = pydicom.sequence.Sequence()

for z, contour_items in dicom_format_coords_by_z.items():
    contour_image_sequence = image_sequence_by_z_collapsed[z]
    for contour_data in contour_items:
        new_contour_dataset = pydicom.dataset.Dataset()
        new_contour_dataset.ContourGeometricType = CONTOUR_GEOMETRIC_TYPE
        if len(contour_data) % 3 != 0:
            raise ValueError("The contour points should be divisible by 3")
        
        new_contour_dataset.NumberOfContourPoints = len(contour_data) // 3
        new_contour_dataset.ContourImageSequence = contour_image_sequence
        new_contour_dataset.ContourData = contour_data
        
        new_contour_sequence.append(new_contour_dataset)
        
    

In [ ]:
new_contour_sequence[0]